# Effectiveness - Add multiple points

In [1]:
import sys
sys.path.append("../..")

from sklearn import svm
import numpy as np
from copy import deepcopy

import src

from src.data_utils import (
    load_npy, save_npy, preprocess_data, variance, normalize, generate_union_description
)

In [2]:
# Read data
x_train, y_train, x_test, y_test, \
columns_name = \
    preprocess_data('train_80p_2c.csv', 'test_80p_2c.csv')

x_train = x_train[:,:2]
x_test = x_test[:,:2]

add_point_idxs = [5, 13]

# pick two points from test for add
add_points_x = x_test[add_point_idxs, :2]
add_points_y = y_test[add_point_idxs]

x_train_added = np.append(x_train[:, :2], add_points_x, axis=0)
y_train_added = np.append(y_train, add_points_y)

x_test_added = np.delete(x_test[:, :2], add_point_idxs, axis=0)
y_test_added = np.delete(y_test, add_point_idxs)

x_test = x_test_added
y_test = y_test_added

union_d = generate_union_description(len(x_train), 10)
union_d_added = deepcopy(union_d)
union_d_added[4] = np.append(union_d_added[4], len(x_train))
union_d_added_once = deepcopy(union_d_added)
union_d_added[9] = np.append(union_d_added[9], len(x_train) + 1)


model = svm.SVC(decision_function_shape='ovo')

mc_plus_ov_82 = src.mc_owen(x_train_added, y_train_added, x_test_added, y_test_added, model, 
                            union_d_added, 1000*len(y_train_added), 4)
save_npy('mc_plus_ov_82.npy', mc_plus_ov_82)

# init ov
model = svm.SVC(decision_function_shape='ovo')

mc_plus_ov_80 = src.mc_owen(x_train, y_train, x_test_added, y_test_added, model,
                            union_d, 1000*len(y_train), 4)
save_npy('mc_plus_ov_80.npy', mc_plus_ov_80)

model = svm.SVC(decision_function_shape='ovo')
mc_ov_82 = src.mc_owen(x_train_added, y_train_added, x_test_added, y_test_added, model,
                       union_d_added, 100*len(y_train_added), 1)
save_npy('mc_ov_82.npy', mc_ov_82)

# delta
init_ov = load_npy('mc_plus_ov_80.npy')

delta_owen = src.DeltaOwen(x_train, y_train, x_test_added, y_test_added, model,
                                init_ov, union_d)
ov_81 = delta_owen.add_single_point(add_points_x[0], add_points_y[0], 4,
                            100*(len(y_train)+1))
delta_owen = src.DeltaOwen(np.append(x_train, [add_points_x[0]], axis=0),
                                np.append(y_train, [add_points_y[0]]),
                                x_test_added, y_test_added, model,
                                ov_81, union_d_added_once)
delta_ov_82 = delta_owen.add_single_point(add_points_x[1], add_points_y[1], 9,
                            100*(len(y_train)+2))

save_npy('delta_ov_82.npy', delta_ov_82)

# pivot
init_ov = load_npy('mc_plus_ov_80.npy')

pivot_owen = src.PivotOwen(x_train, y_train, x_test_added, y_test_added, model,
                              init_ov, union_d)

lov_80 = pivot_owen.prepare(1000*len(y_train))
save_npy('lov_80.npy', lov_80)

ov_81 = pivot_owen.add_single_point(add_points_x[0], add_points_y[0], 4,
                            100*(len(y_train)+1), flags={'flag_lov': True})

pivot_owen.x_train = np.append(x_train, [add_points_x[0]], axis=0)
pivot_owen.y_train = np.append(y_train, add_points_y[0])
pivot_owen.union_description = union_d_added_once

pivot_ov_82 = pivot_owen.add_single_point(add_points_x[1], add_points_y[1], 9,
                                        100*(len(y_train)+2), flags={'flag_lov': True})

save_npy('pivot_ov_82.npy', pivot_ov_82)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')
init_ov = load_npy('mc_plus_sv_80.npy')

# KNN
heur_owen = src.HeurOwen(x_train, y_train,
                              x_test_added, y_test_added,
                              model, init_ov, union_d,
                              params={'method': 'knn'})
heur_owen.prepare(params={'n_neighbors': 3})
knn_ov_82 = heur_owen.add_multi_points(add_points_x, add_points_y, [4, 9])
save_npy('knn_ov_82.npy', knn_ov_82)

# for simi
model = svm.SVC(decision_function_shape='ovo')

idxs = [13, 78]
ovs = list()

for idx in idxs:
    new_union_d = deepcopy(union_d)
    for _ in new_union_d:
        if idx in _:
            _.remove(idx)
    ov = src.mc_owen(np.delete(x_train, idx, axis=0),
                          np.delete(y_train, idx),
                          x_test_added, y_test_added,
                          model, new_union_d, 1000*79, 4)
    save_npy('mc_plus_ov_79_del'+str(idx)+'.npy', ov)
    ovs.append(ov)

ovs = np.asarray(ovs)

save_npy('knn_plus_ovs.npy', ovs)


# KNN+
heur_shap = src.HeurOwen(x_train, y_train,
                              x_test_added, y_test_added,
                              model, init_ov, union_d,
                              params={'method': 'knn+'})

heur_shap.prepare(flags={'exact': False, 'train': False},
                  params={'n_neighbors': 3,
                          'simi_type': 'ed',
                          'f_shap': 'n*n',
                          'rela': ['poly', 1],
                          'train_idxs': [13, 78],
                          'train_svs': load_npy('knn_plus_svs.npy')})

knn_plus_ov_82 = heur_shap.add_multi_points(add_points_x, add_points_y, [4, 9])
save_npy('knn_plus_ov_82.npy', knn_plus_ov_82)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')
init_ov = load_npy('mc_plus_ov_80.npy')
base_shap = src.BaseOwen(x_train, y_train, x_test_added,
                              y_test_added, model, init_ov, union_d)

# local average based
base_ov_82_lavg = base_shap.add_multi_points(add_points_x,
                                            add_points_y,
                                            [4, 9],
                                            params={'method': 'lavg'})

# leave one out based
base_ov_82_gavg = base_shap.add_multi_points(add_points_x,
                                            add_points_y,
                                            [4, 9],
                                            params={'method': 'gavg'})

save_npy('base_ov_82_lavg.npy', base_ov_82_lavg)

save_npy('base_ov_82_gavg.npy', base_ov_82_gavg)

# comparison
def comp(base_v, comp_ov, name):
    var = variance(base_v, normalize(base_v, comp_ov))
    print("variance of %s   \t : %.10f" % (name, var))

mc_plus_ov_82 = load_npy('mc_plus_sv_82.npy')

comp(mc_plus_ov_82, load_npy('delta_ov_82.npy'), 'delta')
comp(mc_plus_ov_82, load_npy('pivot_ov_82.npy'), 'pivot')
comp(mc_plus_ov_82, load_npy('knn_ov_82.npy'), 'knn')
comp(mc_plus_ov_82, load_npy('knn_plus_ov_82.npy'), 'knn+')
comp(mc_plus_ov_82, load_npy('mc_ov_82.npy'), 'mc')
comp(mc_plus_ov_82, load_npy('base_ov_82_lavg.npy'), 'base lavg')
comp(mc_plus_ov_82, load_npy('base_ov_82_gavg.npy'), 'base gavg')